In [ ]:
# tf idf analysis

In [ ]:
pip install scikit-learn nltk

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Load the dataset
cdf = pd.read_csv("Clean_Tweets.csv")  # Adjust path if needed

# Ensure text column is named correctly
text_column = "text"  # Change this if your column has a different name

# Initialize TF-IDF Vectorizer
vectorizer = TfidfVectorizer(stop_words="english", max_features=20)  # Adjust max_features as needed

# Transform the text data into a TF-IDF matrix
tfidf_matrix = vectorizer.fit_transform(df[text_column].astype(str))

# Get feature names (words)
feature_names = vectorizer.get_feature_names_out()

# Convert TF-IDF scores to a DataFrame
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)

# Display top words with highest TF-IDF scores (optional)
print(tfidf_df.sum().sort_values(ascending=False).head(10))  # Top 10 words

# Save the results (optional)
tfidf_df.to_csv("tfidf_results.csv", index=False)

In [ ]:
# linear regression - do negative sentiment scores correlate with frequent words from TF-IDF?

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# Load the TF-IDF results and original dataset
tfidf_df = pd.read_csv("tfidf_results.csv")  # Adjust path if needed
df = pd.read_csv("Clean_Tweets.csv")  # Adjust path if needed

# Ensure text and sentiment columns are named correctly
sentiment_column = "negative_sentiment_score"  # Ensure this exists in your dataset

# Merge the TF-IDF dataframe with the sentiment scores
df_combined = pd.concat([df[sentiment_column], tfidf_df], axis=1)

# Define features (X) and target (y)
X = df_combined.drop(columns=[sentiment_column])  # TF-IDF features
y = df_combined[sentiment_column]  # Negative sentiment scores

# Split into training and test sets (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
r2 = r2_score(y_test, y_pred)
print(f"R² Score: {r2:.4f}")

# Get feature importance (coefficients)
feature_importance = pd.Series(model.coef_, index=X.columns).sort_values(ascending=False)
print("Top 10 most predictive words:")
print(feature_importance.head(10))

# Save feature importance to a file (optional)
feature_importance.to_csv("word_correlation_scores.csv")